PROCESS

In [23]:
# Install needed packages
install.packages("tidyverse", dependencies = TRUE)
install.packages("lubridate")


also installing the dependencies ‘diffobj’, ‘brio’, ‘callr’, ‘digest’, ‘praise’, ‘processx’, ‘ps’, ‘waldo’, ‘covr’, ‘feather’, ‘mockr’, ‘testthat’


Warning message in install.packages("tidyverse", dependencies = TRUE):
“installation of package ‘tidyverse’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
# Combine grouping and summarizing of members and casual day of the week label
combined_data_clean_day_of_week_num_aggregated <- combined_data_clean %>%
  group_by(day_of_week_label) %>%
  summarize(
    #mean_ride_length = mean(ride_length),
    #sum_day_of_week_num = sum(day_of_week_num),
    count = n()
  )

View(combined_data_clean_day_of_week_num_aggregated)

day_of_week_label,count
<ord>,<int>
Sun,743856
Mon,736821
Tue,815719
Wed,837387
Thu,874825
Fri,842731
Sat,898838


In [ ]:
# Combine grouping and summarizing of members and casual day of the week label
combined_data_clean_day_of_week_num_aggregated <- combined_data_clean %>%
  group_by(day_of_week_label) %>%
  summarize(
    #mean_ride_length = mean(ride_length),
    #sum_day_of_week_num = sum(day_of_week_num),
    count = n()
  )

View(combined_data_clean_day_of_week_num_aggregated)

day_of_week_label,count
<ord>,<int>
Sun,743856
Mon,736821
Tue,815719
Wed,837387
Thu,874825
Fri,842731
Sat,898838


In [ ]:
# Combine grouping and summarizing of members and casual day of the week label
combined_data_clean_day_of_week_num_aggregated <- combined_data_clean %>%
  group_by(day_of_week_label) %>%
  summarize(
    #mean_ride_length = mean(ride_length),
    #sum_day_of_week_num = sum(day_of_week_num),
    count = n()
  )

View(combined_data_clean_day_of_week_num_aggregated)

day_of_week_label,count
<ord>,<int>
Sun,743856
Mon,736821
Tue,815719
Wed,837387
Thu,874825
Fri,842731
Sat,898838


In [1]:
# Load the packages
library(readr)
library(dplyr)
library(purrr)
library(tidyr)
library(lubridate)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [2]:
# Set the working directory to the location of the downloaded data
setwd("/home/ladipo/Desktop/Data Analytic/DataSets/Cyclic_Data")

In [3]:
# List all CSV files in the working directory set above
files <- list.files(pattern = "*.csv")

In [4]:
# Read each CSV file and store them in a list for efficient CSV reading
data_list <- lapply(files, read_csv)

Rows: 426590 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (7): ride_id, rideable_type, start_station_name, start_station_id, end_...
dbl  (4): start_lat, start_lng, end_lat, end_lng
dttm (2): started_at, ended_at

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 144873 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (7): ride_id, rideable_type, start_station_name, start_station_id, end_...
dbl  (4): start_lat, start_lng, end_lat, end_lng
dttm (2): started_at, ended_at

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 223164 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (7

In [5]:
# Combine the list of data frames into a single data frame using bind_rows
combined_data <- bind_rows(data_list)

In [6]:
# View the combined data
View(combined_data)

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
<chr>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
8FE8F7D9C10E88C7,electric_bike,2023-04-02 08:37:28,2023-04-02 08:41:37,NA,NA,NA,NA,41.80,-87.60,41.79,-87.60,member
34E4ED3ADF1D821B,electric_bike,2023-04-19 11:29:02,2023-04-19 11:52:12,NA,NA,NA,NA,41.87,-87.65,41.93,-87.68,member
5296BF07A2F77CB5,electric_bike,2023-04-19 08:41:22,2023-04-19 08:43:22,NA,NA,NA,NA,41.93,-87.66,41.93,-87.66,member
40759916B76D5D52,electric_bike,2023-04-19 13:31:30,2023-04-19 13:35:09,NA,NA,NA,NA,41.92,-87.65,41.91,-87.65,member
77A96F460101AC63,electric_bike,2023-04-19 12:05:36,2023-04-19 12:10:26,NA,NA,NA,NA,41.91,-87.65,41.91,-87.63,member
8D6A2328E19DC168,electric_bike,2023-04-19 12:17:34,2023-04-19 12:21:38,NA,NA,NA,NA,41.91,-87.63,41.92,-87.65,member
C97BBA66E07889F9,electric_bike,2023-04-19 09:35:48,2023-04-19 09:45:00,NA,NA,NA,NA,41.93,-87.66,41.91,-87.65,member
6687AD4C575FF734,electric_bike,2023-04-11 16:13:43,2023-04-11 16:18:41,NA,NA,NA,NA,42.00,-87.66,41.99,-87.66,member
A8FA4F73B22BC11F,electric_bike,2023-04-11 16:29:24,2023-04-11 16:40:23,NA,NA,NA,NA,41.99,-87.66,42.00,-87.66,member


In [7]:
# Structure of the data
str(combined_data)

spc_tbl_ [5,750,177 × 13] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ ride_id           : chr [1:5750177] "8FE8F7D9C10E88C7" "34E4ED3ADF1D821B" "5296BF07A2F77CB5" "40759916B76D5D52" ...
 $ rideable_type     : chr [1:5750177] "electric_bike" "electric_bike" "electric_bike" "electric_bike" ...
 $ started_at        : POSIXct[1:5750177], format: "2023-04-02 08:37:28" "2023-04-19 11:29:02" ...
 $ ended_at          : POSIXct[1:5750177], format: "2023-04-02 08:41:37" "2023-04-19 11:52:12" ...
 $ start_station_name: chr [1:5750177] NA NA NA NA ...
 $ start_station_id  : chr [1:5750177] NA NA NA NA ...
 $ end_station_name  : chr [1:5750177] NA NA NA NA ...
 $ end_station_id    : chr [1:5750177] NA NA NA NA ...
 $ start_lat         : num [1:5750177] 41.8 41.9 41.9 41.9 41.9 ...
 $ start_lng         : num [1:5750177] -87.6 -87.7 -87.7 -87.7 -87.7 ...
 $ end_lat           : num [1:5750177] 41.8 41.9 41.9 41.9 41.9 ...
 $ end_lng           : num [1:5750177] -87.6 -87.7 -87.7 -87.7 -87.6 ...
 $ membe

In [8]:
# Convert to datetime using lubridate
combined_data$started_at <- ymd_hms(combined_data$started_at)
combined_data$ended_at <- ymd_hms(combined_data$ended_at)

In [9]:
# Calculate the difference
combined_data$time_diff <- as.duration(interval(combined_data$started_at, combined_data$ended_at))

In [10]:
# Convert the duration to a more readable format, if necessary
combined_data$ride_length <- as.numeric(combined_data$time_diff, units = "mins")

In [11]:
# Extract the day of the week as a number (1 = Sunday, 2 = Monday, ...)
combined_data$day_of_week_num <- wday(combined_data$started_at)

In [12]:
# Extract the day of the week as a label (Sunday, Monday, ...)
combined_data$day_of_week_label <- wday(combined_data$started_at, label = TRUE)

In [16]:
install.packages("geosphere")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [14]:
library(geosphere)

# Calculate distance in meters
combined_data$distance <- distHaversine(p1 = cbind(combined_data$start_lng, combined_data$start_lat), 
                              p2 = cbind(combined_data$end_lng, combined_data$end_lat))

View(combined_data)

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,time_diff,ride_length,day_of_week_num,day_of_week_label,distance
<chr>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<Duration>,<dbl>,<dbl>,<ord>,<dbl>
8FE8F7D9C10E88C7,electric_bike,2023-04-02 08:37:28,2023-04-02 08:41:37,NA,NA,NA,NA,41.80,-87.60,41.79,-87.60,member,249s (~4.15 minutes),4.1500000,1,Sun,1113.1949
34E4ED3ADF1D821B,electric_bike,2023-04-19 11:29:02,2023-04-19 11:52:12,NA,NA,NA,NA,41.87,-87.65,41.93,-87.68,member,1390s (~23.17 minutes),23.1666667,4,Wed,7126.7078
5296BF07A2F77CB5,electric_bike,2023-04-19 08:41:22,2023-04-19 08:43:22,NA,NA,NA,NA,41.93,-87.66,41.93,-87.66,member,120s (~2 minutes),2.0000000,4,Wed,0.0000
40759916B76D5D52,electric_bike,2023-04-19 13:31:30,2023-04-19 13:35:09,NA,NA,NA,NA,41.92,-87.65,41.91,-87.65,member,219s (~3.65 minutes),3.6500000,4,Wed,1113.1949
77A96F460101AC63,electric_bike,2023-04-19 12:05:36,2023-04-19 12:10:26,NA,NA,NA,NA,41.91,-87.65,41.91,-87.63,member,290s (~4.83 minutes),4.8333333,4,Wed,1656.8681
8D6A2328E19DC168,electric_bike,2023-04-19 12:17:34,2023-04-19 12:21:38,NA,NA,NA,NA,41.91,-87.63,41.92,-87.65,member,244s (~4.07 minutes),4.0666667,4,Wed,1995.9922
C97BBA66E07889F9,electric_bike,2023-04-19 09:35:48,2023-04-19 09:45:00,NA,NA,NA,NA,41.93,-87.66,41.91,-87.65,member,552s (~9.2 minutes),9.2000000,4,Wed,2375.4788
6687AD4C575FF734,electric_bike,2023-04-11 16:13:43,2023-04-11 16:18:41,NA,NA,NA,NA,42.00,-87.66,41.99,-87.66,member,298s (~4.97 minutes),4.9666667,3,Tue,1113.1949
A8FA4F73B22BC11F,electric_bike,2023-04-11 16:29:24,2023-04-11 16:40:23,NA,NA,NA,NA,41.99,-87.66,42.00,-87.66,member,659s (~10.98 minutes),10.9833333,3,Tue,1113.1949


In [15]:
# Remove duplicate rows using dplyr
combined_data_unique <- combined_data %>% distinct()

In [16]:
# Filter out rows with NA in the 'started_at' or 'ended_at' columns
combined_data_clean <- combined_data %>%
  filter(!is.na(started_at) & !is.na(ended_at))

View(combined_data_clean)

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,time_diff,ride_length,day_of_week_num,day_of_week_label,distance
<chr>,<chr>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<Duration>,<dbl>,<dbl>,<ord>,<dbl>
8FE8F7D9C10E88C7,electric_bike,2023-04-02 08:37:28,2023-04-02 08:41:37,NA,NA,NA,NA,41.80,-87.60,41.79,-87.60,member,249s (~4.15 minutes),4.1500000,1,Sun,1113.1949
34E4ED3ADF1D821B,electric_bike,2023-04-19 11:29:02,2023-04-19 11:52:12,NA,NA,NA,NA,41.87,-87.65,41.93,-87.68,member,1390s (~23.17 minutes),23.1666667,4,Wed,7126.7078
5296BF07A2F77CB5,electric_bike,2023-04-19 08:41:22,2023-04-19 08:43:22,NA,NA,NA,NA,41.93,-87.66,41.93,-87.66,member,120s (~2 minutes),2.0000000,4,Wed,0.0000
40759916B76D5D52,electric_bike,2023-04-19 13:31:30,2023-04-19 13:35:09,NA,NA,NA,NA,41.92,-87.65,41.91,-87.65,member,219s (~3.65 minutes),3.6500000,4,Wed,1113.1949
77A96F460101AC63,electric_bike,2023-04-19 12:05:36,2023-04-19 12:10:26,NA,NA,NA,NA,41.91,-87.65,41.91,-87.63,member,290s (~4.83 minutes),4.8333333,4,Wed,1656.8681
8D6A2328E19DC168,electric_bike,2023-04-19 12:17:34,2023-04-19 12:21:38,NA,NA,NA,NA,41.91,-87.63,41.92,-87.65,member,244s (~4.07 minutes),4.0666667,4,Wed,1995.9922
C97BBA66E07889F9,electric_bike,2023-04-19 09:35:48,2023-04-19 09:45:00,NA,NA,NA,NA,41.93,-87.66,41.91,-87.65,member,552s (~9.2 minutes),9.2000000,4,Wed,2375.4788
6687AD4C575FF734,electric_bike,2023-04-11 16:13:43,2023-04-11 16:18:41,NA,NA,NA,NA,42.00,-87.66,41.99,-87.66,member,298s (~4.97 minutes),4.9666667,3,Tue,1113.1949
A8FA4F73B22BC11F,electric_bike,2023-04-11 16:29:24,2023-04-11 16:40:23,NA,NA,NA,NA,41.99,-87.66,42.00,-87.66,member,659s (~10.98 minutes),10.9833333,3,Tue,1113.1949


In [17]:
# Export dataframe to CSV file on my computer
write.csv(combined_data_clean, file = "/home/ladipo/Desktop/Data Analytic/Coursera Training/combined_data_clean_bike_vscode.csv", row.names = FALSE)

AGGREGATION

In [17]:
# Combine grouping and summarizing of members and casual day of the week label
combined_data_clean_day_of_week_num_aggregated <- combined_data_clean %>%
  group_by(day_of_week_label) %>%
  summarize(
    #mean_ride_length = mean(ride_length),
    #sum_day_of_week_num = sum(day_of_week_num),
    count = n()
  )

View(combined_data_clean_day_of_week_num_aggregated)

day_of_week_label,count
<ord>,<int>
Sun,743856
Mon,736821
Tue,815719
Wed,837387
Thu,874825
Fri,842731
Sat,898838


In [20]:
# Group by 'day_of_week_label' and count 'member' and 'casual'
member_casual_aggregated <- combined_data_clean %>%
  dplyr::group_by(day_of_week_label) %>%
  dplyr::summarize(
    count_member = sum(member_casual == "member"),
    count_casual = sum(member_casual == "casual")
  )

View(member_casual_aggregated)

day_of_week_label,count_member,count_casual
<ord>,<int>,<int>
Sun,407528,336328
Mon,500519,236302
Tue,571565,244154
Wed,590031,247356
Thu,601874,272951
Fri,530154,312577
Sat,480379,418459


In [21]:
# Group by 'rideable_type' and count 'member' and 'casual'
rideable_type_aggregated <- combined_data_clean %>%
  dplyr::group_by(rideable_type) %>%
  dplyr::summarize(
    count_member = sum(member_casual == "member"),
    count_casual = sum(member_casual == "casual")
  )

View(rideable_type_aggregated)

rideable_type,count_member,count_casual
<chr>,<int>,<int>
classic_bike,1868311,905858
docked_bike,0,71334
electric_bike,1813739,1090935
